In [2]:
import pandas as pd
import sklearn 
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np
from sklearn import feature_selection
import seaborn as sns

In [5]:
#Fundo a ser analisado
nome_do_fundo = 'paiva'
coldata = "data"
nome_benchmark = "ibov"

In [8]:
df=pd.read_csv('P:\\ciencia_de_dados\\Correlacao_de_fundos\\Desenvolvimento\\1DataPrep\\base_'+ nome_do_fundo + '.csv',sep =';',decimal=',',index_col=0)

In [9]:
df

,Product,FinancialPrice,data,MesAno,Mes,Ano,Retorno_1,Retorno
0,paiva,4.470000,2021-08-12,08-2021,8,2021,NaN,NaN
1,paiva,4.483246,2021-08-11,08-2021,8,2021,0.997045,-0.295456
2,paiva,4.501608,2021-08-10,08-2021,8,2021,0.995921,-0.407899
3,paiva,4.527160,2021-08-09,08-2021,8,2021,0.994356,-0.564416
4,paiva,4.514663,2021-08-06,08-2021,8,2021,1.002768,0.276809
...,...,...,...,...,...,...,...,...
3449,ibov,48000.900000,2015-01-06,01-2015,1,2015,1.030458,3.045776
3450,ibov,47516.800000,2015-01-05,01-2015,1,2015,1.010188,1.018798
3451,ibov,48512.200000,2015-01-02,01-2015,1,2015,0.979481,-2.051855
3452,ibov,50007.400000,2015-01-01,01-2015,1,2015,0.970100,-2.989957


In [10]:
df_fundo = df[df.Product == nome_do_fundo]

In [11]:
#Retorno Mensal
pivot_table = pd.pivot_table(df_fundo,index=['Ano'],values=["Retorno_1"],fill_value=1,aggfunc='prod',columns=['Mes'])#,margins=True,margins_name="Year")
pivot_table = (pivot_table * 100 - 100)#.iloc[:-1]

In [12]:
# backgroung color mapping
print("\nDataframe - Gradient color:")
from  matplotlib.colors import LinearSegmentedColormap
my_cmap=LinearSegmentedColormap.from_list('rg',["r", "w", "g"], N=256) 

pd.options.display.float_format = "{:,.3f}".format

pivot_table.style.background_gradient(cmap=my_cmap,vmin = -20,vmax = 20)


Dataframe - Gradient color:


In [13]:
#Retorno Anualizado (Media Geometrica dos retornos anuais vinda de precos diarios)
retorno_anualizado = (df_fundo["Retorno_1"].product()**(252/float(df_fundo["Product"].count())) - 1) * 100
retorno_anualizado

16.978895602283274

In [14]:
#Volatilidade anualizada
vol_anualizada = df_fundo["Retorno"].std() * (252**(1/2))
vol_anualizada

21.522976579490905

In [15]:
#Sharpe
retorno_anualizado / vol_anualizada #.67 

0.7888730231887324

In [16]:
#Max DrawDown historico
x=df.FinancialPrice[df.Product == nome_do_fundo]
max(x)/float(min(x))-1

2.2044468583501122

In [17]:
#Max drawdown mensal
df.groupby(['MesAno','Product']).agg({"FinancialPrice":  lambda x: max(x)/float(min(x))-1})

FinancialPrice
MesAno  Product                
01-2015 ibov              0.066
        paiva             0.032
01-2016 ibov              0.156
        paiva             0.058
01-2017 ibov              0.111
...                         ...
12-2018 paiva             0.032
12-2019 ibov              0.076
        paiva             0.102
12-2020 ibov              0.072
        paiva             0.050

[162 rows x 1 columns]

In [18]:
#Descritivas mensais
#df[['Product','MesAno']].groupby(['MesAno','Product']).agg(['mean', 'min', 'max','std','count'])
df.groupby(['Product']).describe()
df.groupby(['MesAno','Product']).describe()

FinancialPrice                                                \
                         count        mean       std         min         25%   
MesAno  Product                                                                
01-2015 ibov            22.000  48,443.764   824.496  46,907.700  47,790.825   
        paiva           22.000       1.494     0.012       1.479       1.484   
01-2016 ibov            21.000  39,514.171 1,758.294  37,497.500  38,031.200   
        paiva           21.000       1.504     0.024       1.471       1.486   
01-2017 ibov            22.000  63,638.409 1,791.775  59,588.700  62,086.200   
...                        ...         ...       ...         ...         ...   
12-2018 paiva           21.000       2.617     0.027       2.581       2.592   
12-2019 ibov            22.000 113,136.682 2,621.767 108,927.800 110,967.225   
        paiva           22.000       3.860     0.125       3.653       3.743   
12-2020 ibov            23.000 115,976.048 2,551.449 111,399.900 113,771.650   
        paiva           23.000       4.193     0.073       4.083       4.127   

                                                       Mes         ...  \
                        50%         75%         max  count   mean  ...   
MesAno  Product                                                    ...   
01-2015 ibov     48,325.950  48,972.450  50,007.400 22.000  1.000  ...   
        paiva         1.492       1.500       1.526 22.000  1.000  ...   
01-2016 ibov     38,944.400  40,612.200  43,350.000 21.000  1.000  ...   
        paiva         1.498       1.518       1.557 21.000  1.000  ...   
01-2017 ibov     63,952.400  64,633.400  66,190.600 22.000  1.000  ...   
...                     ...         ...         ...    ...    ...  ...   
12-2018 paiva         2.606       2.638       2.662 21.000 12.000  ...   
12-2019 ibov    112,590.300 115,645.300 117,203.200 22.000 12.000  ...   
        paiva         3.864       3.987       4.025 22.000 12.000  ...   
12-2020 ibov    116,148.600 117,940.550 119,409.200 23.000 12.000  ...   
        paiva         4.209       4.252       4.288 23.000 12.000  ...   

                Retorno_1       Retorno                                    \
                      75%   max   count   mean   std    min    25%    50%   
MesAno  Product                                                             
01-2015 ibov        1.009 1.030  22.000 -0.205 1.717 -2.990 -1.700 -0.087   
        paiva       1.003 1.013  22.000 -0.091 0.718 -1.406 -0.475 -0.089   
01-2016 ibov        1.007 1.046  21.000 -0.300 1.771 -2.789 -1.523 -0.203   
        paiva       1.005 1.023  21.000 -0.047 0.934 -2.084 -0.724 -0.040   
01-2017 ibov        1.008 1.037  22.000  0.391 1.226 -2.623 -0.291  0.269   
...                   ...   ...     ...    ...   ...    ...    ...    ...   
12-2018 paiva       1.005 1.022  21.000  0.009 0.826 -1.516 -0.309  0.000   
12-2019 ibov        1.006 1.015  22.000  0.274 0.613 -0.763 -0.007  0.145   
        paiva       1.009 1.013  22.000  0.402 0.546 -0.482  0.000  0.325   
12-2020 ibov        1.009 1.019  23.000  0.291 0.826 -1.865 -0.071  0.179   
        paiva       1.006 1.014  23.000  0.209 0.644 -1.026 -0.023  0.122   

                             
                  75%   max  
MesAno  Product              
01-2015 ibov    0.928 3.046  
        paiva   0.318 1.257  
01-2016 ibov    0.660 4.597  
        paiva   0.548 2.296  
01-2017 ibov    0.762 3.734  
...               ...   ...  
12-2018 paiva   0.550 2.217  
12-2019 ibov    0.644 1.509  
        paiva   0.867 1.320  
12-2020 ibov    0.853 1.883  
        paiva   0.608 1.420  

[162 rows x 40 columns]